# Analyzing Trends in Education Within Sub-Saharn Africa

## Importing packages

In [2]:
# import relevant packages here or going forward
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from   sqlalchemy.engine import URL
from   sqlalchemy import text

### Connecting to SQL server

In [17]:
# Write your answer here: 
url_server = URL.create(
    "postgresql",
    host = 'localhost',
    database = 'postgres',
    username = 'postgres',
    port = 5432,
    password  = "postgres") # I installed postgres in a separate class so I have a password

connection = create_engine(url_server)

### Loading the Data

In [ ]:
education_data = pd.read_csv('data/wdi_education_data.csv')

### Pivoting
After I (Alix) was unable to safely pivot the table in postgres, I did it in python and reuploaded it back to sql

In [23]:
# Load the data
education_data = pd.read_csv('data/wdi_education_data.csv')

# Step 1: Select only the year columns (starting from the 4th column onward)
year_columns = education_data.columns[3:]

# Step 2: Remove rows with NaN values in the year columns
education_data_cleaned = education_data.dropna(subset=year_columns)

# Step 3: Melt the DataFrame to convert year columns into rows
education_data_melted = education_data_cleaned.melt(
    id_vars=["Country Name", "Series Name"],  # Keep 'Country Name' and 'Series Name' as they are
    value_vars=year_columns,  # The year columns (from column 4 onward)
    var_name="Year",  # New column for year
    value_name="Value"  # New column for values
)

# Step 4: Clean up the 'Year' column by extracting the year (if necessary)
education_data_melted["Year"] = education_data_melted["Year"].str.extract(r"(\d{4})")

# Step 5: Remove rows where 'Year' is NaN before converting to integer
education_data_melted = education_data_melted.dropna(subset=["Year"])

# Step 6: Convert the 'Year' column to integer
education_data_melted["Year"] = education_data_melted["Year"].astype(int)

# Step 7: Remove duplicate rows in the melted data
education_data_melted = education_data_melted.drop_duplicates(subset=["Country Name", "Year", "Series Name"])

# Pivot the data using the `pivot_table` method
education_data_pivoted = education_data_melted.pivot_table(
    index=["Country Name", "Year"],  # Use both 'Country Name' and 'Year' as index
    columns="Series Name",  # Pivot on 'Series Name'
    values="Value",  # Use 'Value' as the values to fill in the pivoted table
    aggfunc="first"  # Aggregation function in case of duplicate rows, here we take the first
)

# Reset the index to move 'Country Name' and 'Year' into regular columns
education_data_pivoted_reset = education_data_pivoted.reset_index()

# Step 8: Ensure unique column names by renaming duplicates
education_data_pivoted_reset.columns = [str(col) + "_" + str(i) if education_data_pivoted_reset.columns.tolist().count(col) > 1 else col for i, col in enumerate(education_data_pivoted_reset.columns)]

# Print out the columns
print("Columns in the pivoted DataFrame:")
print(education_data_pivoted_reset.columns)

# Step 8: Check for uniqueness
duplicate_columns = education_data_pivoted_reset.columns[education_data_pivoted_reset.columns.duplicated()]

if len(duplicate_columns) > 0:
    print("\nDuplicate columns found:")
    print(duplicate_columns)
else:
    print("\nAll column names are unique.")


KeyboardInterrupt: 

In [24]:
# Step 8: Check for uniqueness
duplicate_columns = education_data_pivoted_reset.columns[education_data_pivoted_reset.columns.duplicated()]

if len(duplicate_columns) > 0:
    print("\nDuplicate columns found:")
    print(duplicate_columns)
else:
    print("\nAll column names are unique.")


All column names are unique.


### Uploading the Data into SQL

In [26]:
education_data_pivoted_reset.to_sql('wdi_education_data',
               con = connection,
               if_exists='replace',
               index=False)


ProgrammingError: (psycopg2.errors.DuplicateColumn) column "Educational attainment, Doctoral or equivalent, population 25+," specified more than once

[SQL: 
CREATE TABLE wdi_education_data (
	"Country Name" TEXT, 
	"Year" BIGINT, 
	"Adjusted net enrollment rate, primary (%% of primary school age children)" TEXT, 
	"Adjusted net enrollment rate, primary, female (%% of primary school age children)" TEXT, 
	"Adjusted net enrollment rate, primary, male (%% of primary school age children)" TEXT, 
	"Adolescents out of school (%% of lower secondary school age)" TEXT, 
	"Adolescents out of school, female (%% of female lower secondary school age)" TEXT, 
	"Adolescents out of school, male (%% of male lower secondary school age)" TEXT, 
	"Children out of school (%% of primary school age)" TEXT, 
	"Children out of school, female (%% of female primary school age)" TEXT, 
	"Children out of school, male (%% of male primary school age)" TEXT, 
	"Children out of school, primary" TEXT, 
	"Children out 